## To do

#### - Merge do dataset de AQI com o dataset de meteorologia ✅
#### - Visualização de dados (gráficos, box plots, matriz de correlação, outliers) ✅
#### - Teste de hipóteses
#### - Feature Engineering

### - Prever com algoritmos o AQI baseado em dados meteorológicos em determinadas coordenadas ✅

## Trabalho Futuro

#### - Interpolação noutras coordenadas
#### - Data augmentation

In [6]:
import pandas as pd 

In [7]:
# Read the data again

df = pd.read_csv('datasets/FINALDATASET.csv')

In [3]:
# Find the coordinates to interpolate

# import numpy as np

# Define the coordinates

# coordinates = [
#     [41.5549, -8.4067],
#     [41.653315, -8.58779],
#     [41.650791, -8.43569],
#     [41.449721999999994, -8.296389]
# ]

# Convert the coordinates to a numpy array
# coordinates_array = np.array(coordinates)

# Calculate the mean of the latitude and longitude
# mean_latitude = np.mean(coordinates_array[:, 0])
# mean_longitude = np.mean(coordinates_array[:, 1])

# Print the medium point
# print("Medium Point Latitude:", mean_latitude)
# print("Medium Point Longitude:", mean_longitude)

# 41.57718199999999,-8.43164225 - ZONA NOVA ARCADA
# https://www.google.com/maps/place/41%C2%B034'37.9%22N+8%C2%B025'53.9%22W/@41.5772226,-8.4318715,53m/data=!3m1!1e3!4m4!3m3!8m2!3d41.577182!4d-8.4316422

In [5]:
# from scipy.interpolate import Rbf
# 
# filtered_df = df[df['parameter'].isin(['pm1', 'pm25', 'pm10','no2'])]
# 
# coordinates = filtered_df[['latitude', 'longitude']].values
# values = filtered_df['value'].values

# Perform IDW interpolation
# rbf = Rbf(coordinates[:, 0], coordinates[:, 1], values)
# interpolated_value = rbf(41.577929249999995, -8.40629225)

# Print the interpolated value
# print("Interpolated Value:", interpolated_value)

# Linear Regression - Primeira abordagem

In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Carrega o conjunto de dados
df = pd.read_csv('datasets/FINALDATASET.csv')

# Separando os dados de entrada (X) e saída (y)
X = df.drop('AQI', axis=1)
y = df['AQI']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definindo as colunas numéricas e categóricas
numeric_features = ['longitude', 'latitude', 'no2', 'pm10', 'pm25', 'pm1',
                    'tempmax', 'tempmin', 'temp', 'feelslike', 'dew',
                    'humidity', 'severerisk', 'precip', 'snow', 'windspeed',
                    'sealevelpressure', 'cloudcover', 'visibility',
                    'solarradiation', 'solarenergy', 'uvindex']
categorical_features = ['date', 'time']

# Pipeline de pré-processamento
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean'))])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Modelagem e treinamento
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])
model.fit(X_train, y_train)

# Avaliação do modelo
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Cálculo do erro quadrático médio
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print("Erro Quadrático Médio (conjunto de treinamento):", mse_train)
print("Erro Quadrático Médio (conjunto de teste):", mse_test)


Erro Quadrático Médio (conjunto de treinamento): 1013.889873019862
Erro Quadrático Médio (conjunto de teste): 1179.4432989023771


# Random Forest - Primeira abordagem

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# Selecionar as features e o target
features = ['longitude', 'latitude', 'no2', 'pm10', 'pm25', 'pm1', 'tempmax', 'tempmin', 'temp', 'feelslike', 'dew', 'humidity', 'severerisk', 'precip', 'snow', 'windspeed', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex']
target = 'AQI'

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Pré-processamento: preencher valores ausentes com a média
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Criar e treinar o modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões no conjunto de treinamento e teste
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcular o erro quadrático médio
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print("Erro Quadrático Médio (conjunto de treinamento):", mse_train)
print("Erro Quadrático Médio (conjunto de teste):", mse_test)

Erro Quadrático Médio (conjunto de treinamento): 0.004191248860528708
Erro Quadrático Médio (conjunto de teste): 5.7994791666667735e-05
